# Report and plots `fig:threshold__tradeoff`

In [ ]:
import os
import tqdm

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter("ignore")

<br>

## Prepare the report

In [ ]:
PREFIX = "appendix__"

report = "./grids/musicnet__threshold.pk"

experiments = [
# # 1.5/200  # the best model
#     './grids/grid-fast/musicnet-fast__00/musicnet[003]-098',  # ARD
#     './grids/grid-fast/musicnet-fast__01/musicnet[001]-046',
#     './grids/grid-fast/musicnet-fast__02/musicnet[004]-124',
#     './grids/grid-fast/musicnet-fast__03/musicnet[002]-072',
#     './grids/grid-fast/musicnet-fast__04/musicnet[000]-020',

#     './grids/grid-fast/musicnet-fast__00/musicnet[003]-085',  # VD
#     './grids/grid-fast/musicnet-fast__01/musicnet[001]-033',
#     './grids/grid-fast/musicnet-fast__02/musicnet[004]-111',
#     './grids/grid-fast/musicnet-fast__03/musicnet[002]-059',
#     './grids/grid-fast/musicnet-fast__04/musicnet[000]-007',

# 1/200
    './grids/grid-fast/musicnet-fast__00/musicnet[003]-097',  # ARD
    './grids/grid-fast/musicnet-fast__01/musicnet[001]-045',
    './grids/grid-fast/musicnet-fast__02/musicnet[004]-123',
    './grids/grid-fast/musicnet-fast__03/musicnet[002]-071',
    './grids/grid-fast/musicnet-fast__04/musicnet[000]-019',

    './grids/grid-fast/musicnet-fast__00/musicnet[003]-084',  # VD
    './grids/grid-fast/musicnet-fast__01/musicnet[001]-032',
    './grids/grid-fast/musicnet-fast__02/musicnet[004]-110',
    './grids/grid-fast/musicnet-fast__03/musicnet[002]-058',
    './grids/grid-fast/musicnet-fast__04/musicnet[000]-006',
    
# # 1/2000
#     './grids/grid-fast/musicnet-fast__00/musicnet[003]-093',  # ARD
#     './grids/grid-fast/musicnet-fast__01/musicnet[001]-041',
#     './grids/grid-fast/musicnet-fast__02/musicnet[004]-119',
#     './grids/grid-fast/musicnet-fast__03/musicnet[002]-067',
#     './grids/grid-fast/musicnet-fast__04/musicnet[000]-015',

#     './grids/grid-fast/musicnet-fast__00/musicnet[003]-080',  # VD
#     './grids/grid-fast/musicnet-fast__01/musicnet[001]-028',
#     './grids/grid-fast/musicnet-fast__02/musicnet[004]-106',
#     './grids/grid-fast/musicnet-fast__03/musicnet[002]-054',
#     './grids/grid-fast/musicnet-fast__04/musicnet[000]-002',

# 1/20
    './grids/grid-fast/musicnet-fast__00/musicnet[003]-101',  # ARD
    './grids/grid-fast/musicnet-fast__01/musicnet[001]-049',
    './grids/grid-fast/musicnet-fast__02/musicnet[004]-127',
    './grids/grid-fast/musicnet-fast__03/musicnet[002]-075',
    './grids/grid-fast/musicnet-fast__04/musicnet[000]-023',

    './grids/grid-fast/musicnet-fast__00/musicnet[003]-088',  # VD
    './grids/grid-fast/musicnet-fast__01/musicnet[001]-036',
    './grids/grid-fast/musicnet-fast__02/musicnet[004]-114',
    './grids/grid-fast/musicnet-fast__03/musicnet[002]-062',
    './grids/grid-fast/musicnet-fast__04/musicnet[000]-010',
]

<br>

In [ ]:
assert False, '''Run cells below to create an analyzer script for this experiment.'''

<br>

Pick a name for the report pickle and compile a **bash** script for
building the threshold figure for each of epxeriment in the list above.

In [ ]:
import stat

devspec = """--devices "cuda:0" "cuda:1" "cuda:2" "cuda:3" --per-device 3"""

bash = "./grids/musicnet__threshold.sh"
with open(bash, "w") as fout:
    for experiment in experiments:
        path = os.path.abspath(os.path.normpath(experiment))
        fout.write(f"""python -m cplxpaper.auto.reports {devspec} --append"""
                   f""" "threshold" "{os.path.abspath(report)}" "{path}.json"\n""")

# allow exc and keep r/w
os.chmod(bash, stat.S_IXUSR | stat.S_IRUSR | stat.S_IWUSR)

bash

<br>

In [ ]:
assert False, '''Run all below to make the figure.'''

<br>

## Build the table

Load the report constructed on the selected experiments.

In [ ]:
from cplxpaper.auto.reports.utils import restore
from cplxpaper.auto.parameter_grid import reconstruct_grid

def build_report(filename):
    report = tqdm.tqdm(restore(filename), desc="analyzing report data")
    workers, results = zip(*report)    
    if not results:
        return {}, []

    # compute the grid and flatten the manifests
    experiments, thresholds, options, *results = zip(*results)
    full_grid, flat_options = reconstruct_grid(options)

    return full_grid, [*zip(experiments, thresholds, flat_options, *results)]

Extract the score from the scorers' output.

In [ ]:
from cplxpaper.auto.reports.utils import dict_get_one

def get_score(score):
    n_zer, n_par = map(sum, zip(*score["sparsity"].values()))
    return {
        "score": dict_get_one(score, "pooled_average_precision", "accuracy"),
        "compression": n_par / (n_par - n_zer)
    }

Evaluate several grids and join them

In [ ]:
raw_grid, output = build_report(report)

Alter the recovered grid

In [ ]:
grid = set(field for field in raw_grid
           if not any(map(field.__contains__, {
                # service fields
                "__name__", "__timestamp__", "__version__", "device",

                # ignore global model class settings
                "model__cls",

                # upcast is a service variable, which only complex models have
                #  and it is usually mirrored in `features` settings.
                "__upcast"
            })))

grid.update({
    "stages__sparsify__model__cls",
    "stages__sparsify__objective__kl_div",
    "threshold"  # ensure threshold is included
})

Index by the experiment **grid--folder** and prepare fields

In [ ]:
experiments, thresholds, options, *rest = zip(*output)

# experiment paths are absolute!
df = pd.DataFrame([*zip(experiments, thresholds)], columns=["experiment", "threshold"])
df = df["experiment"].str.replace(os.path.commonpath(experiments), "*")\
                     .str.extract("^(?P<grid>.*)/(?P<experiment>[^/]*)$", expand=True)\
                     .join(df["threshold"])

master_index = df.set_index(["grid", "experiment", "threshold"]).index

Gradually construct the table of options

In [ ]:
parameters = pd.DataFrame(index=master_index)

Assign proper tags to models

In [ ]:
from cplxpaper.auto.reports.utils import get_model_tag

def patched_get_model_tag(opt):
    tag = get_model_tag(opt)

    # Legacy model patch: if not specified then True (see `musicnet.models.base`)
    cls = tag["model"]
    if "DeepConvNet" in cls and opt.get("model__legacy", True):
        cls += " k3"

    return {**tag, "model": cls}

grid = [k for k in grid if not k.startswith((
    "model__",
    "stages__sparsify__model__"
))]

parameters = parameters.join(pd.DataFrame([
    *map(patched_get_model_tag, options)
], index=master_index))

Other fields' preprocessing.

In [ ]:
assert 'dataset' not in grid
assert 'features' not in grid

Only the essential experiment parameters should have remained by now.

In [ ]:
parameters = parameters.join(pd.DataFrame([
    {g: opt[g] for g in grid} for opt in options
], index=master_index))

grid

Now collect the metrics. We need:
* **accuracy** performance on `dense`, `pre-fine-tune` and `post-fine-tune`
* **compression rate** from a `fine-tune` stage

In [ ]:
scores, *tail = rest
assert not tail

metrics = pd.DataFrame([
    get_score(dict_get_one(score, "test", "test-256")) for score in scores
], index=master_index)

Join the tables and rename unfotunate columns.

In [ ]:
df_main = parameters.join(metrics).rename(columns={
    "stages__sparsify__objective__kl_div": "kl_div"
})

<br>

## Create the threshold plot

Decide on the target folder and computation cache.

In [ ]:
report_name = "figure__musicnet__threshold"

report_target = os.path.normpath(os.path.abspath(os.path.join(
    "../../assets", report_name
)))

A service plotting function to darkern the specified colour

In [ ]:
from matplotlib.ticker import FormatStrFormatter, FuncFormatter


def darker(color, a=0.5):
    """Adapted from this stackoverflow question_.
    .. _question: https://stackoverflow.com/questions/37765197/
    """
    from matplotlib.colors import to_rgb
    from colorsys import rgb_to_hls, hls_to_rgb

    h, l, s = rgb_to_hls(*to_rgb(color))
    return hls_to_rgb(h, max(0, min(a * l, 1)), s)

Group by all fileds except for `threshold`:
* `model`, `kind`, `method`, `dataset`, `features` and `kl_div`

In [ ]:
print([f for f in parameters.columns if "kl_div" not in f])
fields = [
    'method',
    'model',
    'kind',
    'kl_div'
]

Handle colours

In [ ]:
def kind_model_method_color(kind, model, method, kl_div):
    return {  # VD/ARD
        # tab10 colours are paired! use this to keep similar models distinguishable
        ("C"  , "DeepConvNet",   "VD", 1/200): "C0",
        ("C"  , "DeepConvNet",  "ARD", 1/200): "C1",
        ("C"  , "DeepConvNet",   "VD", 1/20): "C2",
        ("C"  , "DeepConvNet",  "ARD", 1/20): "C3",
        ("C"  , "DeepConvNet",   "VD", 1/2000): "C4",
        ("C"  , "DeepConvNet",  "ARD", 1/2000): "C5",
    }[kind, model, method, kl_div]

Make a crude plot

In [ ]:
fig, (ax_l, ax_r) = plt.subplots(2, 1, figsize=(8, 5), dpi=300, sharex=True)
fig.patch.set_alpha(1.0)

ax_l.set_title("The effect of $\\tau$ on performance and compression (MusicNet)")

# set up limits and axis labels
ax_l.set_ylabel("Average Precision")
ax_r.set_ylabel("$\\times$ compression")
ax_r.set_yscale("log")

ax_r.set_xlabel("Threshold $\\tau$")

ax_l.set_xlim(-3.6125, 3.6125)
ax_l.set_ylim(0.55, 0.75)
ax_r.set_ylim(40, 2000)

# Trabelsi et al. (2018)
ax_l.axhline(0.729, color="k", alpha=0.25, zorder=-10, lw=1)
ax_l.annotate("Trabelsi et al. (2018)", xy=(0, 0.75),  xycoords='data',
              xytext=(0.05, 0.935), textcoords='axes fraction', alpha=0.75)


# group by tau and experiment spec and plot
grouper = df_main.groupby(fields)
for key, df in tqdm.tqdm(grouper, desc="populating plots"):
    df = df[["score", "compression"]].sort_index()
    label = dict(zip(fields, key))

    m, min_, max_ = df.mean(level=-1), df.min(level=-1), df.max(level=-1)
    color = kind_model_method_color(**label)

    for ax, field, marker in zip([ax_l, ax_r], ["score", "compression"], ["", "o"]):
        ax.fill_between(m.index, min_[field], max_[field],
                          color=darker(color, 1.4), alpha=0.25, zorder=10)
        ax.plot(m[field], c=color, alpha=1.0, marker=marker, markersize=4,
                label="{kind} {model} {method} ($C={kl_div}$)".format(**label),
                zorder=15)

ax_l.legend(ncol=1, loc=(0.55, .05))  # loc="center right")

ax_l.axvline(-0.5, c="k", lw=2, zorder=2)
ax_r.axvline(-0.5, c="k", lw=2, zorder=2)
# ax_r.grid(axis='y', which='both')
# ax_l.grid(axis='y', which='both')

plt.tight_layout(h_pad=-0.55)
fig.savefig(f"{report_target}.pdf", dpi=300)

# plt.show()
plt.close()

In [ ]:
assert False

<br>